In [146]:
import open3d as o3d
import numpy as np

In [147]:
def convert_point_cloud_to_numpy_array(point_cloud : o3d.geometry.PointCloud) -> np.ndarray:
    return np.asarray(point_cloud.points)

In [148]:
def convert_numpy_to_point_cloud(numpy_array : np.ndarray) -> o3d.geometry.PointCloud:
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(numpy_array)
    return pcd

In [149]:
def write_zeros_in_file(count_of_points : int, path_to_new_label_file : str):
    with open(path_to_new_label_file, 'w') as file:
        file.write('0\n' * count_of_points)

In [150]:
def select_points_from_point_cloud_by_label_id(point_cloud : o3d.geometry.PointCloud, path_to_label_file : str, label_id : int) -> o3d.geometry.PointCloud:
    labels = np.fromfile(path_to_label_file, dtype=np.uint32)
    labels = labels.reshape((-1))
    pcd_point_by_id = point_cloud.select_by_index(np.where(labels == label_id)[0])
    return pcd_point_by_id

In [151]:
def segment_plane_from_point_cloud(point_cloud : o3d.geometry.PointCloud, distance : float = 0.1) -> (o3d.geometry.PointCloud, o3d.geometry.PointCloud):
    try:
        _, inliers = point_cloud.segment_plane(distance_threshold=distance,
                                             ransac_n=3,
                                             num_iterations=5000)
    except Exception:
        return (point_cloud, point_cloud.clear())
    inlier_cloud = point_cloud.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([1.0, 0, 0])
    outlier_cloud = point_cloud.select_by_index(inliers, invert=True)
    return (inlier_cloud, outlier_cloud)

In [152]:
def extract_point_cloud_from_bin_file(path_to_bin_file : str) -> o3d.geometry.PointCloud:
    pcd_np = np.fromfile(path_to_bin_file, dtype=np.float32).reshape(-1, 4)[:, :3]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_np[:, :3])
    return pcd

In [153]:
def create_point_cloud_by_label_list(point_cloud : o3d.geometry.PointCloud, path_to_label_file : str, label_list : list) -> o3d.geometry.PointCloud:
    numpy_arr = np.empty((0,3), float)
    for label in label_list:
        current_point_cloud = select_points_from_point_cloud_by_label_id(point_cloud, path_to_label_file, label)
        converted_to_numpy_point_cloud = convert_point_cloud_to_numpy_array(current_point_cloud)
        numpy_arr = np.append(numpy_arr, converted_to_numpy_point_cloud, axis=0)
    return convert_numpy_to_point_cloud(numpy_arr)

In [154]:
def segment_all_planes_from_point_cloud(point_cloud : o3d.geometry.PointCloud) -> list:
    all_planes = []
    inlier_cloud, outlier_cloud = scan_next_plane_from_point_cloud(point_cloud, 0.3)
    while outlier_cloud.has_points():
        all_planes.append(inlier_cloud)
        inlier_cloud, outlier_cloud = scan_next_plane_from_point_cloud(outlier_cloud, 0.3)
    if (inlier_cloud.has_points()): all_planes.append(inlier_cloud)
    
    return all_planes

In [163]:
def get_indexes_of_points(main_point_cloud : np.ndarray, temp_point_cloud : np.ndarray) -> list:
    index_list = []
    for point in temp_point_cloud:
        for index, main_point in enumerate(main_point_cloud):
            if (np.allclose(point, main_point, rtol=1e-10)):
                index_list.append(index)
    return index_list

In [156]:
def write_ones_to_file_by_index(index_list : list, path_to_new_label_file : str):
    file = open(path_to_new_label_file, 'r')
    list_strings = f.readlines()
    file.close()
    for index in index_list:
        list_strings[index] = "1\n"
    file = open(path_to_new_label_file, 'w')
    file.writelines(list_strings)
    file.close()

In [157]:
def work_with_file(point_cloud : o3d.geometry.PointCloud, path_to_label_file : str, label_list : list, path_to_new_label_file : str):
    main_point_cloud = extract_point_cloud_from_bin_file(current_map)
    numpy_main_point_cloud = convert_point_cloud_to_numpy_array(main_point_cloud)
    write_zeros_in_file(len(numpy_main_point_cloud), path_to_new_label_file)
    current_point_cloud = create_point_cloud_by_label_list(main_point_cloud, path_to_label_file, PLANE_LIST)
    planes_list = segment_all_planes_from_point_cloud(current_point_cloud)
    
    for plane in planes_list:
        numpy_plane = convert_point_cloud_to_numpy_array(plane)
        index_list = get_indexes_of_points(numpy_main_point_cloud, numpy_plane)
        write_ones_to_file_by_index(index_list, path_to_new_label_file)

In [158]:
PATH_TO_BIN_FOLDER = "/home/pavel/dataset/sequences/00/velodyne/"
PATH_TO_LABEL_FOLDER = "/home/pavel/dataset/sequences/00/labels/"
PATH_TO_NEW_LABEL_BINARY_FILE = "/home/pavel/Point-Cloud/src/label000000.txt"

In [159]:
ROAD_LABEL = 40 # - plane
SIDEWALK_LABEL = 48 # - plane
PARKING_LABEL = 44 # - plane
OTHER_GROUND_LABEL = 49 # - poka xz (na pervoy .pcd ih prosto net)
BUILDING_LABEL = 50 # - plane
OTHER_STRUCTURE_LABEL = 52 # - not plane
VEGETATION_LABEL = 70 # - not plane
TRUNK_LABEL = 71 # - not plane
TERRAIN_LABEL = 72 # - plane
FENCE_LABEL = 51 # - plane

### Функция должна выглядеть примерно как-то так:

In [160]:
current_map = PATH_TO_BIN_FOLDER + "000000.bin"
current_label_file = PATH_TO_LABEL_FOLDER + "000000.label"

In [161]:
PLANE_LIST = [ROAD_LABEL]

In [164]:
work_with_file(current_map, current_label_file, PLANE_LIST, PATH_TO_NEW_LABEL_BINARY_FILE)

KeyboardInterrupt: 